### **ПРАКТИЧЕСКОЕ ЗАДАНИЕ**

**Задание**

Взять тот же датасет, который был на вебинаре и предобученную модель для задачи суммаризации
1. Проверить насколько хорошо она суммаризирует
2.(дополнительно) Сделать генерацию заголовков для статьи (обучить модель для генерации заголовков)



In [ ]:
# Импортируем библиотеки

import json
import random
!pip install transformers
!pip install rouge
from transformers import AutoTokenizer, MBartForConditionalGeneration, MBartTokenizer
!pip install transformers sentencepiece
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

In [2]:
# Загрузим датасет

!wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
# !wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
# !wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

# # по сути данные те же можно пользоваться любым способом загрузки
# from datasets import load_dataset
# dataset = load_dataset('IlyaGusev/gazeta', revision="v1.0")["test"]

In [3]:
# Посмотрим на длину датасета 

# !head -n 1 gazeta_train.txt
!cat gazeta_train.txt | wc -l
# !cat gazeta_val.txt | wc -l
# !cat gazeta_test.txt | wc -l

52400


In [4]:
# Функция извлечения статей из файла

def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

In [5]:
# Извлечем статьи

train_records = read_gazeta_records("gazeta_train.txt")
# val_records = read_gazeta_records("gazeta_val.txt")
# test_records = read_gazeta_records("gazeta_test.txt")

In [6]:
train_records[0]

{'url': 'https://www.gazeta.ru/financial/2011/11/30/3852658.shtml',
 'text': '«По итогам 2011 года чистый отток может составить примерно $80 млрд, в следующем году — около $20 млрд. При этом мы ожидаем, что со второго полугодия 2012 года начнется приток капитала», — заявил «Интерфаксу» замминистра экономического развития Андрей Клепач. Официальные прогнозы по выводу капитала из России становятся все пессимистичными: еще летом власти полагали, что из страны уйдет не более $35 млрд, в сентябре Минэкономразвития назвал цифру $50 млрд, в начале ноября Центробанк пересмотрел оценку до $70 млрд. Очередное изменение прогноза было ожидаемо: по расчетам Центробанка , за январь — октябрь чистый отток капитала достиг $64 млрд, причем в последние месяцы он ускорился: в сентябре он составил $14 млрд, в октябре — $13 млрд против среднего ежемесячного оттока в $6—8 млрд в первом полугодии. «После октябрьских данных Минэкономразвития вынуждено было изменить оценку, настаивать на $70 млрд означало ожид

In [7]:
# Разделим текст статей и краткое их содержание

texts = [txt['text'] for txt in train_records]
print(texts[0])

summaries = [summ['summary'] for summ in train_records]
print(summaries[0])

«По итогам 2011 года чистый отток может составить примерно $80 млрд, в следующем году — около $20 млрд. При этом мы ожидаем, что со второго полугодия 2012 года начнется приток капитала», — заявил «Интерфаксу» замминистра экономического развития Андрей Клепач. Официальные прогнозы по выводу капитала из России становятся все пессимистичными: еще летом власти полагали, что из страны уйдет не более $35 млрд, в сентябре Минэкономразвития назвал цифру $50 млрд, в начале ноября Центробанк пересмотрел оценку до $70 млрд. Очередное изменение прогноза было ожидаемо: по расчетам Центробанка , за январь — октябрь чистый отток капитала достиг $64 млрд, причем в последние месяцы он ускорился: в сентябре он составил $14 млрд, в октябре — $13 млрд против среднего ежемесячного оттока в $6—8 млрд в первом полугодии. «После октябрьских данных Минэкономразвития вынуждено было изменить оценку, настаивать на $70 млрд означало ожидать серьезного замедления оттока капитала на непонятно каких причинах», — гово

#### **1. СУММАРИЗАЦИЯ ТЕКСТОВ**

In [8]:
# Загрузим токенайзер и модель

model_name = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name, src_lang="ru_RU")
model = MBartForConditionalGeneration.from_pretrained(model_name).cuda()

In [9]:
print(type(tokenizer), type(model))

<class 'transformers.models.mbart.tokenization_mbart_fast.MBartTokenizerFast'> <class 'transformers.models.mbart.modeling_mbart.MBartForConditionalGeneration'>


In [10]:
# Функция суммаризации текста

def get_summary(data, tokenizer, model):
  preds = []
  for item in range(len(data)):
    input_ids = tokenizer([data[item]],
                          return_tensors="pt",
                          padding="max_length",
                          truncation=True,
                          max_length=600)["input_ids"].cuda()

    output_ids = model.generate(input_ids=input_ids,
                                max_length=162,
                                no_repeat_ngram_size=3,
                                num_beams=8,
                                top_k=0)[0]

    summary = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

    preds.append(summary)

  return preds

In [11]:
# Bleu measures precision: how much the words (and/or n-grams) in the machine generated summaries appeared in the human reference summaries.
# Bleu Scores are between 0 and 1. A score of 0.6 or 0.7 is considered the best you can achieve. 
# Even two humans would likely come up with different sentence variants for a problem, and would rarely achieve a perfect match. 
# For this reason, a score closer to 1 is unrealistic in practice and should raise a flag that your model is overfitting.

# Rouge measures recall: how much the words (and/or n-grams) in the human reference summaries appeared in the machine generated summaries.
# A Rouge score close to 0 indicates poor similarity between candidate and references. A Rouge score close to 1 indicates strong similarity between candidate and references.


# Функция подсчета метрик

def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        bleu_score = corpus_bleu([[r] for r in references], predictions)
        print("BLEU: ", bleu_score)
    if metric in ("rouge", "all"):
        rouge = Rouge()
        rouge_scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", rouge_scores)
    
    return bleu_score, rouge_scores

In [12]:
# Проэкспериментируем на одной статье

result = get_summary([texts[0]], tokenizer, model)

print(texts[0])
print(result[0])

score = calc_scores([summaries[0]], result, metric="all")

# BLEU:  0.2845596039624504
# ROUGE:  {'rouge-1': {'f': 0.23255813470795034, 'p': 0.2857142857142857, 'r': 0.19607843137254902}, 'rouge-2': {'f': 0.06666666196543243, 'p': 0.08823529411764706, 'r': 0.05357142857142857}, 'rouge-l': {'f': 0.10898670282455002, 'p': 0.14285714285714285, 'r': 0.09803921568627451}}

«По итогам 2011 года чистый отток может составить примерно $80 млрд, в следующем году — около $20 млрд. При этом мы ожидаем, что со второго полугодия 2012 года начнется приток капитала», — заявил «Интерфаксу» замминистра экономического развития Андрей Клепач. Официальные прогнозы по выводу капитала из России становятся все пессимистичными: еще летом власти полагали, что из страны уйдет не более $35 млрд, в сентябре Минэкономразвития назвал цифру $50 млрд, в начале ноября Центробанк пересмотрел оценку до $70 млрд. Очередное изменение прогноза было ожидаемо: по расчетам Центробанка , за январь — октябрь чистый отток капитала достиг $64 млрд, причем в последние месяцы он ускорился: в сентябре он составил $14 млрд, в октябре — $13 млрд против среднего ежемесячного оттока в $6—8 млрд в первом полугодии. «После октябрьских данных Минэкономразвития вынуждено было изменить оценку, настаивать на $70 млрд означало ожидать серьезного замедления оттока капитала на непонятно каких причинах», — гово

In [13]:
# Посмотрим на метрики на 10-ти статьях

predictions = get_summary(texts[:10], tokenizer, model)
mtrcs = calc_scores(summaries[:10], predictions, metric="all")

# BLEU:  0.5046505343003475
# ROUGE:  {'rouge-1': {'f': 0.32963202606351644, 'p': 0.35747931737008065, 'r': 0.3140353716072964}, 'rouge-2': {'f': 0.1695545968622948, 'p': 0.1831540307598914, 'r': 0.16350571756918506}, 'rouge-l': {'f': 0.28757367976279447, 'p': 0.320460964325129, 'r': 0.28445367631352686}}

Count: 10
Ref: На приближающихся парламентских и президентских выборах граждане России больше не смогут работать в международных наблюдательных миссиях. Такое решение принял в четверг ЦИК. Российским членам делегаций нельзя ограничить и въезд в страну, и перемещения по ней, а иностранным — можно, указывают эксперты.
Hyp: Центризбирком принял постановление, ограничивающее работу международных наблюдателей на выборах депутатов Госдумы в декабре 2011-го и президента РФ в марте 2012-го. Эксперты считают, что нововведение может существенно повлиять как на результат работы наблюдателей, так и на репутацию российских выборов.
BLEU:  0.5236472489372311
ROUGE:  {'rouge-1': {'f': 0.3165010984110964, 'p': 0.3263598555224705, 'r': 0.32209106000731763}, 'rouge-2': {'f': 0.16111144578497735, 'p': 0.1652412227697331, 'r': 0.1670771461406136}, 'rouge-l': {'f': 0.27628431918341906, 'p': 0.29849546266808635, 'r': 0.2936226013361104}}


In [14]:
summaries[:10]

['В 2011 году из России уйдет $80 млрд, считают в Минэкономразвития. Менее месяца назад Центробанк давал оценку $70 млрд, повысив первоначальный прогноз вдвое. Отток капитала из страны усиливается из-за кризиса в Европе, а в декабре российским компаниям выплачивать внешние долги. На движение капитала повлияли и выборы: несмотря на их предсказуемость, бизнес хочет ясности с перестановками в правительстве.',
 'Юлия Соловьева, экс-директор холдинга «Профмедиа», возглавила российский офис Google. Поиск американская интернет-корпорация вела полгода. Медиаменеджер с опытом работы в международных компаниях найдет общий язык с акционерами, но прорыва можно было ждать от технического специалиста или маркетолога, говорят аналитики.',
 'Суд арестовал на два месяца четверых экс-чиновников правительства Дагестана, включая бывшего врио премьер-министра Дагестана Абдусамада Гамидова. Все они обвиняются в мошенничестве. Защита фигурантов просила поместить подсудимых под домашний арест или отпустить по

In [15]:
predictions

['В 2011 году из России утекло $80 млрд, прогнозирует Минэкономразвития. Это один из самых высоких за последние 20 лет. Причина — нестабильность на внешних финансовых рынках и рост опасений относительно второй волны рецессии. Внутри страны эксперты не видят особых причин для бегства капитала.',
 'Российское подразделение Google возглавила Юлия Соловьева, экс-директор московского офиса международной консалтинговой компании Alvarez & Marsal. В новой должности она будет отвечать за стратегическое развитие компании в России, «на одном из крупнейших европейских интернет-рынков». Ранее источники говорили о том, что в качестве руководителя российского подразделения рассматривался внутренний иностранный кандидат, а также представители Apple, Hewlett-Packard, Microsoft, «Сколково».',
 'Басманный районный суд Москвы арестовал по обвинению в мошенничестве четверых бывших чиновников правительства Дагестана — врио премьер-министра Абдусамада Гамидова, его заместителей Шамиль Исаева и Раюдина Юсуфов

#### **ВЫВОД:**

В целом какие-то моменты модель улавливает, а иногда выделяет как важное совсем не то, что было указано в summary-метке (при визуальной оценке текста). Стоит более детально поподбирать параметры. Например, увеличение параметра **num_beams** позволило улучшить метрики и Bleu и Rouge. Увеличение максимальной длины (как изначального текста, так и краткого содержания) дало прирост в меньшей степени, чем только изменение num_beams.

#### **2. ГЕНЕРАЦИЯ ЗАГОЛОВКОВ**

In [16]:
# Выделим заголовки из датасета

titles = [txt['title'] for txt in train_records]
print(titles[0])

Прогноз не успевает за оттоком


In [17]:
# Функция генерации заголовков

def get_title(data, tokenizer, model):
  preds = []
  for item in range(len(data)):
    input_ids = tokenizer([data[item]],
                          return_tensors="pt",
                          padding="max_length",
                          truncation=True,
                          max_length=800)["input_ids"].cuda()

    output_ids = model.generate(input_ids=input_ids,
                                max_length=15,
                                no_repeat_ngram_size=1,
                                num_beams=8,
                                top_k=0,
                                # temperature=5
                                )[0]

    title = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

    preds.append(title)

  return preds

In [18]:
# Сделаем предсказания и посмотрим на метрики

tt_predictions = get_title(titles[:10], tokenizer, model)
tt_mtrcs = calc_scores(titles[:10], tt_predictions, metric="all")

Count: 10
Ref: Россиян исключили из наблюдения
Hyp: Russia исключили из наблюдения.
BLEU:  0.45926533035332284
ROUGE:  {'rouge-1': {'f': 0.4349999966867285, 'p': 0.4252777777777778, 'r': 0.4699999999999999}, 'rouge-2': {'f': 0.35380952089972795, 'p': 0.35190476190476183, 'r': 0.3733333333333333}, 'rouge-l': {'f': 0.4036507936504698, 'p': 0.40861111111111115, 'r': 0.4533333333333333}}


In [19]:
titles[:10]

['Прогноз не успевает за оттоком',
 'Google закончил поиск',
 '«Фигуранты дела могут давить на свидетелей»',
 '«С последних традиционно «отжимают» больше»',
 'Третий «Голос» за Градского',
 '«О сборе биоматериала россиян сообщили спецслужбы»',
 'Банки снизят ставки по инерции',
 '«Это не «Динамо», а ХК МВД»',
 '«Игра получилась и без «стольника»',
 'Россиян исключили из наблюдения']

In [20]:
tt_predictions

['forecast не успевает за оттоком',
 'Android Google объявил о завершении поиска по запросу «Google»',
 'Westhamp.com опубликовал видеозапись,',
 'Stateside.com опубликовал видеоролик, в котором',
 'Three «Голос» за Градского в третий раз подряд',
 'gingсменты сообщили о сборе биоматериала россиян',
 'Bankи снизят ставки по инерции',
 'Stateside.com опубликовал видеоролик, в котором',
 '«Игра получилась и без«стольника»',
 'Russia исключили из наблюдения.']

#### **ВЫВОД:**

Заголовки модель генерирует гораздо лучше. Интересно, что первые слова почему-то генерируются на английском языке, хотя у модели стоит параметр исходный язык - русский. Заголовок номер 9 - стопроцентное попадание.

Для повышения метрики были уменьшены параметры **no_repeat_ngram_size, max_length (генерации), num_beams**. Изменение параметров: **max_length, temperature** - никак не повлияло на результат.

### **Полезные ссылки**

#### **mBART huggingface**
* BART paper: https://arxiv.org/abs/1910.13461
* mBART paper: https://arxiv.org/abs/2001.08210
* Model: https://huggingface.co/IlyaGusev/mbart_ru_sum_gazeta

[Метрики bleu и rouge](https://stackoverflow.com/questions/38045290/text-summarization-evaluation-bleu-vs-rouge)

[Метрики bleu и rouge precision](https://stats.stackexchange.com/questions/502064/shouldnt-rouge-1-precision-be-equal-to-bleu-with-w-1-0-0-0-when-brevity-pe)